# 作業 : (Kaggle)鐵達尼生存預測
***
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [16]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor

data_path = '../data/Part02/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)

df_bak = df.copy()

df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [21]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
df = df_bak.copy()
df['Cabin'] = df['Cabin'].fillna('None')
embark_mean = df.groupby(['Cabin'])['Fare'].mean().reset_index()
embark_median = df.groupby(['Cabin'])['Fare'].median().reset_index()
embark_mode = df.groupby(['Cabin'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
embark_max = df.groupby(['Cabin'])['Fare'].max().reset_index()
temp = pd.merge(embark_mean,embark_median, on=['Cabin'], how='left')
temp = pd.merge(temp,embark_mode, on=['Cabin'], how='left')
temp = pd.merge(temp,embark_max, on=['Cabin'], how='left')
temp.columns = ['Cabin','Cabin_mean','Cabin_median','Cabin_mode','Cabin_max']
df = pd.merge(df, temp, on=['Cabin'], how='left')

df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cabin_mean,Cabin_median,Cabin_mode,Cabin_max
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S,19.157325,10.5000,8.0500,512.3292
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,71.283300,71.2833,71.2833,71.2833
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S,19.157325,10.5000,8.0500,512.3292
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,53.100000,53.1000,53.1000,53.1000
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S,19.157325,10.5000,8.0500,512.3292


In [22]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

9 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin_mean', 'Cabin_median', 'Cabin_mode', 'Cabin_max']



,Pclass,Age,SibSp,Parch,Fare,Cabin_mean,Cabin_median,Cabin_mode,Cabin_max
0,3,22.0,1,0,7.2500,19.157325,10.5000,8.0500,512.3292
1,1,38.0,1,0,71.2833,71.283300,71.2833,71.2833,71.2833
2,3,26.0,0,0,7.9250,19.157325,10.5000,8.0500,512.3292
3,1,35.0,1,0,53.1000,53.100000,53.1000,53.1000,53.1000
4,3,35.0,0,0,8.0500,19.157325,10.5000,8.0500,512.3292


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

> AND：結果反而不好，分數有些微下降。0.7004864906462793 -->> 0.6937385696153646

In [23]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
# 沒有這四個新特徵的 dataframe 稱為 df_minus
df_minus = df.drop(['Cabin_mean', 'Cabin_median', 'Cabin_mode', 'Cabin_max'],axis=1)

train_X = MMEncoder.fit_transform(df_minus.astype('float'))
estimator = LogisticRegression(solver='lbfgs')
print(f'Score for 原始特徵 + 邏輯斯迴歸 : {cross_val_score(estimator,train_X,train_Y,cv=5).mean()}')

Score for 原始特徵 + 邏輯斯迴歸 : 0.7004864906462793


In [24]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df.astype('float'))
print(f'Score for 新特徵 + 邏輯斯迴歸 : {cross_val_score(estimator,train_X,train_Y,cv=5).mean()}')

Score for 新特徵 + 邏輯斯迴歸 : 0.6937385696153646


---

In [25]:
# 原始特徵 + 梯度提升樹
train_X = MMEncoder.fit_transform(df_minus.astype('float'))
estimator = GradientBoostingRegressor()
print(f'Score for 原始特徵 + 梯度提升樹 : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')

Score for 原始特徵 + 梯度提升樹 : 0.1824165009851612


In [26]:
# 新特徵 + 梯度提升樹
train_X = MMEncoder.fit_transform(df.astype('float'))
print(f'Score for 新特徵 + 梯度提升樹 : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')

Score for 新特徵 + 梯度提升樹 : 0.16579061378362842
